In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from tqdm import tqdm
import csv
import pandas as pd
import re
import os
import json
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain


# from threading import Thread
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda')

In [3]:
# 加载模型
model_cache_path = r'autodl-tmp/Qwen/Qwen2___5-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=model_cache_path,
        trust_remote_code=True,
    )
model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_cache_path,
        # device_map='balanced_low_0',
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        temperature=0
        ).to(device)
model = model.eval()
model.generation_config = GenerationConfig.from_pretrained(
    model_cache_path,
    trust_remote_code=True
)
# 可指定不同的生成长度、top_p等相关超参
processor = AutoProcessor.from_pretrained(model_cache_path)
print('模型加载完成！')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型加载完成！


In [4]:
datas = pd.read_excel('意图识别数据集.xlsx')
questions = datas['问题']
labels = datas['问题分类']
# sublabel = set(labels.values.tolist())
file = '实体提取_改进2.csv'
for question, label in tqdm(zip(questions, labels), ncols=len(questions)) :
    template = f"""
            请分析用户输入的问题，理解用户的需求。根据要求进行输出。不用展示分析过程。参考example进行识别。

            1-分析输入问题的文本和语法结构。
            2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。
            3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。
            4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。
            5-识别出的词以逗号分隔。
            6-根据输出格式输出。

            example:
                问题：肌肉关节智能运动康复评估训练系统有哪些品牌？
                识别结果：肌肉关节智能运动康复评估训练系统，品牌

                问题：流式细胞仪的供应商省份是哪里？
                识别结果：流式细胞仪，供应商省份

                问题：《财政部关于做好政府采购信息公开工作的通知》的来源是什么？
                识别结果：《财政部关于做好政府采购信息公开工作的通知》，来源

                问题：2020年11月发布了多少个政策文件？
                识别结果：2020年11月，发布，政策文件

                问题：“中国福利会国际和平妇幼保健院2024年医疗设备（四）”项目的采购人名称是什么？
                识别结果：“中国福利会国际和平妇幼保健院2024年医疗设备（四）”项目，采购人名称
                

            如果识别结果只有一个词，那么需要进一步分析是否存在主题词、意图词、关键词。
            特别注意不要把较长的主语漏掉，这个也是实体。
            主语词一般在开头的第一个‘的’字前。

            
            输出格式：
                识别结果：

            文本：{question}
            """
            #问题主干
            # 不存在实体输出格式：
            #     实体提取: [],
            #     意图分类:日常问题
                # 实体提取和意图分类结果：

    messages = [
                # {'role':'system', 'content': template},
                {'role':'user', 'content': template}
                ]
    # print('打印messages内容：', messages)
    # print('打印messages类型：', type(messages))
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors='pt').to(device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('question:', question)
    print('response', response)
    body_ = response.split('识别结果：')[-1].replace(' ','').rsplit(',')
    data_ = [question, body_, label]
    with open(file, 'a', newline='', encoding='utf-8') as f:
        fw = csv.writer(f)
        if not os.path.getsize(file):
            header = ['问题', '提取的实体', '问题分类']
            fw.writerow(header)
        fw.writerow(data_)
    # print(f'问题：{question}', f'意图分类：{target}', f'标签真值：{label}')


0it [00:00, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
1it [00:00,  1.59it/s]

question: 医用DR设备都有哪些品牌？
response 识别结果：医用DR设备，品牌


2it [00:01,  1.74it/s]

question: 动态三维脊柱姿态测量分析装置北京的供应商有哪些品牌？
response 识别结果：动态三维脊柱姿态测量分析装置，供应商，品牌


3it [00:01,  1.96it/s]

...

question: 无轨迹等张肌力测试康复训练系统的品牌有哪些？
response 识别结果：无轨迹等张肌力测试康复训练系统，品牌


479it [03:34,  2.39it/s]

question: 无轨迹等张肌力测试康复训练系统的产品价格范围是多少？
response 识别结果：无轨迹等张肌力测试康复训练系统，产品价格范围


480it [03:34,  2.23it/s]

question: 无轨迹等张肌力测试康复训练系统的供应商城市有哪些？
response 识别结果：无轨迹等张肌力测试康复训练系统，供应商城市


In [5]:
# example:
#     问题：肌肉关节智能运动康复评估训练系统有哪些品牌？
#     识别结果：肌肉关节智能运动康复评估训练系统（主语词），品牌（意图词）

#     问题：流式细胞仪的供应商省份是哪里？
#     识别结果：流式细胞仪（设备词），供应商省份（意图词）

In [147]:
data_

,0
0,全自动发药机的供应商地址在哪里？
1,日常问题
2,商品数据
3,"[全自动发药机, 供应商, 地址]"


In [ ]:
template = f"""
            你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
            
            任务：
            1-判断用户问题是否存在实体。
            2-抽取用户问题中可能表达意图的实体。
            3-根据实体与给出的意图标签进行判定该用户问题的意图。
            
            工作流程：
            1-先判断是否存在实体，不存在实体则实体提取为[]，意图分类为日常问题，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
            2-实体提取：请从用户问题中提取出所有实体，多个实体请用逗号分隔。
            3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类用户问题。

            输出格式：
                实体提取：实体
                意图分类：意图标签


            问题：{question}。

            对比实体提取的结果，筛掉问题中不存在的、重复的实体。实体与实体之间是否用逗号分隔，请矫正。

            请根据意图分类结果与用户问题进行分析，再次根据提取的实体对用户问题给出正确的意图标签。
            
            每个问题只能分为一类意图，矫正后的意图分类结果替换前面的分类结果。
            
            请根据生成的结果与要求的输出格式对比，存在格式问题则进行矫正后输出。

            意图标签：商品数据，招标中标信息，政策内容，政策文件信息，日常问题。

            实体提取和意图分类结果："""

In [ ]:
template = f"""
            你是一名实体提取领域专家，请严格遵循以下任务和工作流程的指示输出结果。
            
            任务：
            1-判断上下文是否存在实体。
            2-抽取上下文中表达意图的所有实体。
            
            工作流程：
            1-首先对输入文本进行分词。
            1-其次判断每个分词是否为实体。
            2-判断结果表明所有分词不是实体则直接输出实体提取：''。
            3-判断每个分词，存在至少一个实体的，则根据输出格式提取并输出。
            4-请从上下文中提取出所有实体，多个实体请用逗号分隔。

            对比实体提取的结果，筛掉上下文中不存在的、重复的实体。实体与实体之间是否用逗号分隔，请矫正。

            请根据生成的结果与要求的输出格式对比，存在格式问题则进行矫正后输出。

            输出格式：
                实体提取：实体

            请注意输出结果严格按照输出格式进行，不需要输出分析过程。

            上下文：{question}"""

In [ ]:
template = f"""
            请根据问题进行实体提取，尽可能提取所有实体，按照格式输出。
            请检查提取的实体和问题，所有实体是否占据问题较小篇幅，是则进一步辨别实体并提取。
            可能存在较长实体，请提取出来。
            请将产品名称、产品系统等实体也提取出来。按照格式输出。
            提取出的实体存在重复的请筛掉。
            请将产品名称、产品系统等实体也提取出来。按照格式输出。

            输出格式：
                实体提取：实体，实体，

            问题：{question}
            """